## CBoW মডেল প্রশিক্ষণ

এই নোটবুকটি [AI for Beginners Curriculum](http://aka.ms/ai-beginners)-এর একটি অংশ।

এই উদাহরণে, আমরা CBoW ভাষা মডেল প্রশিক্ষণ করার মাধ্যমে আমাদের নিজস্ব Word2Vec এমবেডিং স্পেস তৈরি করার প্রক্রিয়া দেখব। আমরা টেক্সটের উৎস হিসেবে AG News ডেটাসেট ব্যবহার করব।


In [30]:
from tensorflow import keras
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

আমরা ডেটাসেট লোড করা শুরু করব:


In [1]:
ds_train, ds_test = tfds.load('ag_news_subset').values()

## CBoW মডেল

CBoW $2N$ প্রতিবেশী শব্দের ভিত্তিতে একটি শব্দ অনুমান করতে শেখে। উদাহরণস্বরূপ, যখন $N=1$, আমরা বাক্য *I like to train networks* থেকে নিম্নলিখিত জোড়াগুলি পাব: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks)। এখানে, প্রথম শব্দটি প্রতিবেশী শব্দ যা ইনপুট হিসেবে ব্যবহৃত হয়, এবং দ্বিতীয় শব্দটি যা আমরা অনুমান করছি।

পরবর্তী শব্দ অনুমান করার জন্য একটি নেটওয়ার্ক তৈরি করতে, আমাদের প্রতিবেশী শব্দ ইনপুট হিসেবে সরবরাহ করতে হবে এবং শব্দ নম্বর আউটপুট হিসেবে পেতে হবে। CBoW নেটওয়ার্কের স্থাপত্য নিম্নরূপ:

* ইনপুট শব্দটি এমবেডিং লেয়ারের মাধ্যমে প্রেরণ করা হয়। এই এমবেডিং লেয়ারটি আমাদের Word2Vec এমবেডিং হবে, তাই আমরা এটিকে আলাদাভাবে `embedder` ভেরিয়েবল হিসেবে সংজ্ঞায়িত করব। এই উদাহরণে আমরা এমবেডিং সাইজ = 30 ব্যবহার করব, যদিও আপনি উচ্চতর মাত্রা নিয়ে পরীক্ষা করতে পারেন (বাস্তব Word2Vec-এ 300 থাকে)।
* এমবেডিং ভেক্টরটি এরপর একটি ডেন্স লেয়ারে প্রেরণ করা হবে যা আউটপুট শব্দ অনুমান করবে। সুতরাং এতে `vocab_size` নিউরন থাকবে।

Keras-এ এমবেডিং লেয়ার স্বয়ংক্রিয়ভাবে সংখ্যাসূচক ইনপুটকে এক-হট এনকোডিংয়ে রূপান্তর করতে জানে, তাই আমাদের আলাদাভাবে ইনপুট শব্দ এক-হট এনকোড করার প্রয়োজন নেই। আমরা `input_length=1` নির্দিষ্ট করি যাতে বোঝানো যায় যে আমরা ইনপুট সিকোয়েন্সে কেবল একটি শব্দ চাই - সাধারণত এমবেডিং লেয়ার দীর্ঘ সিকোয়েন্সের সাথে কাজ করার জন্য ডিজাইন করা হয়।

আউটপুটের জন্য, যদি আমরা `sparse_categorical_crossentropy` কে লস ফাংশন হিসেবে ব্যবহার করি, তবে আমাদের কেবল শব্দ নম্বরগুলো প্রত্যাশিত ফলাফল হিসেবে সরবরাহ করতে হবে, এক-হট এনকোডিং ছাড়াই।

আমরা `vocab_size` 5000 সেট করব যাতে গণনার পরিমাণ কিছুটা সীমিত করা যায়। আমরা একটি ভেক্টরাইজারও সংজ্ঞায়িত করব যা আমরা পরে ব্যবহার করব।


In [68]:
vocab_size = 5000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))
embedder = keras.layers.Embedding(vocab_size,30,input_length=1)

model = keras.Sequential([
    embedder,
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1, 30)             150000    
                                                                 
 dense_1 (Dense)             (None, 1, 5000)           155000    
                                                                 
Total params: 305,000
Trainable params: 305,000
Non-trainable params: 0
_________________________________________________________________


চলুন ভেক্টরাইজারটি আরম্ভ করি এবং শব্দভাণ্ডারটি বের করি:


In [69]:
def extract_text(x):
    return x['title']+' '+x['description']

vectorizer.adapt(ds_train.take(500).map(extract_text))
vocab = vectorizer.get_vocabulary()

## প্রশিক্ষণের ডেটা প্রস্তুত করা

এখন আমরা প্রধান ফাংশনটি প্রোগ্রাম করব, যা টেক্সট থেকে CBoW শব্দ জোড়া তৈরি করবে। এই ফাংশনটি আমাদের উইন্ডো সাইজ নির্দিষ্ট করার সুযোগ দেবে এবং একটি জোড়ার সেট - ইনপুট এবং আউটপুট শব্দ - ফেরত দেবে। লক্ষ্য করুন, এই ফাংশনটি শব্দের উপর যেমন কাজ করতে পারে, তেমনি ভেক্টর/টেনসরের উপরও কাজ করতে পারে - যা আমাদের টেক্সট এনকোড করতে দেবে, `to_cbow` ফাংশনে পাঠানোর আগে।


In [70]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(vectorizer('I like to train networks')))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[<tf.Tensor: shape=(), dtype=int64, numpy=376>, <tf.Tensor: shape=(), dtype=int64, numpy=771>], [<tf.Tensor: shape=(), dtype=int64, numpy=3>, <tf.Tensor: shape=(), dtype=int64, numpy=771>], [<tf.Tensor: shape=(), dtype=int64, numpy=771>, <tf.Tensor: shape=(), dtype=int64, numpy=376>], [<tf.Tensor: shape=(), dtype=int64, numpy=3>, <tf.Tensor: shape=(), dtype=int64, numpy=376>], [<tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=376>], [<tf.Tensor: shape=(), dtype=int64, numpy=771>, <tf.Tensor: shape=(), dtype=int64, numpy=3>], [<tf.Tensor: shape=(), dtype=int64, numpy=376>, <tf.Tensor: shape=(), dtype=int64, numpy=3>], [<tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=3>]

চলুন প্রশিক্ষণের ডেটাসেট প্রস্তুত করি। আমরা সমস্ত খবরের মধ্য দিয়ে যাব, `to_cbow` কল করে শব্দ জোড়ার তালিকা পাব, এবং সেই জোড়াগুলো `X` এবং `Y` তে যোগ করব। সময়ের স্বার্থে, আমরা শুধুমাত্র প্রথম ১০k খবর বিবেচনা করব - আপনি সহজেই এই সীমাবদ্ধতা সরিয়ে দিতে পারেন যদি আপনার অপেক্ষা করার জন্য বেশি সময় থাকে এবং আরও ভালো এম্বেডিং পেতে চান :)


In [100]:
X = []
Y = []
for i,x in zip(range(10000),ds_train.map(extract_text).as_numpy_iterator()):
    for w1, w2 in to_cbow(vectorizer(x),window_size=1):
        X.append(tf.expand_dims(w1,0))
        Y.append(tf.expand_dims(w2,0))

আমরা সেই ডেটাকে একটিমাত্র ডেটাসেটে রূপান্তর করব এবং প্রশিক্ষণের জন্য ব্যাচ করব:


In [101]:
ds = tf.data.Dataset.from_tensor_slices((X,Y)).batch(256)

এখন আসল প্রশিক্ষণ শুরু করা যাক। আমরা `SGD` অপ্টিমাইজার ব্যবহার করব, যার লার্নিং রেট বেশ উচ্চ। আপনি `Adam` এর মতো অন্যান্য অপ্টিমাইজার নিয়ে পরীক্ষা করতে পারেন। আমরা শুরুতে ২০০ এপোকের জন্য প্রশিক্ষণ করব - এবং যদি আপনি আরও কম ক্ষতি চান তবে এই সেলটি পুনরায় চালাতে পারেন।


In [102]:
model.compile(optimizer=keras.optimizers.SGD(lr=0.1),loss='sparse_categorical_crossentropy')
model.fit(ds,epochs=200)

Epoch 1/200


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


2156/2156 [==============================] - 7s 3ms/step - loss: 5.6134
Epoch 2/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.5431
Epoch 3/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.5029
Epoch 4/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4754
Epoch 5/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4548
Epoch 6/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4382
Epoch 7/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4243
Epoch 8/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4123
Epoch 9/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4019
Epoch 10/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.3926
Epoch 11/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.3843
Epoch 12/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.3767
E

## Word2Vec চেষ্টা করা

Word2Vec ব্যবহার করতে, চলুন আমাদের শব্দভাণ্ডারের সমস্ত শব্দের জন্য ভেক্টর বের করি:


In [103]:
vectors = embedder(vectorizer(vocab))
vectors = tf.reshape(vectors,(-1,30)) # we need reshape to get rid of extra dimension

চলুন দেখি, উদাহরণস্বরূপ, কীভাবে **Paris** শব্দটি একটি ভেক্টরে এনকোড করা হয়:


In [104]:
paris_vec = embedder(vectorizer('paris'))[0]
print(paris_vec)

tf.Tensor(
[-0.13308628  0.50972325  0.00344684  0.185389   -0.03176536  0.22262476
 -0.3856765  -0.6854793   0.5185803  -0.7215402  -0.16101503  0.15622072
  0.00653811 -0.14954254  0.03379822 -0.01243829  0.27907634 -0.32538188
  0.21718933  0.31112966 -0.24142407  0.15589055  0.2915561   0.19029242
  0.08425518 -0.0941902  -0.54313695 -0.24854654  0.26196313  0.18027727], shape=(30,), dtype=float32)


ওয়ার্ড২ভেক ব্যবহার করে সমার্থক শব্দ খোঁজা বেশ আকর্ষণীয়। নিম্নলিখিত ফাংশনটি একটি নির্দিষ্ট ইনপুটের জন্য `n` সংখ্যক নিকটতম শব্দ ফেরত দেবে। এগুলো খুঁজে বের করার জন্য, আমরা $|w_i - v|$ এর নর্ম গণনা করি, যেখানে $v$ আমাদের ইনপুট শব্দের সাথে সম্পর্কিত ভেক্টর এবং $w_i$ হলো শব্দভাণ্ডারের $i$-তম শব্দের এনকোডিং। এরপর আমরা অ্যারে সাজাই এবং `argsort` ব্যবহার করে সংশ্লিষ্ট সূচকগুলো ফেরত দিই, এবং তালিকার প্রথম `n` উপাদান নিই, যা শব্দভাণ্ডারে নিকটতম শব্দগুলোর অবস্থান এনকোড করে।


In [105]:
def close_words(x,n=5):
  vec = embedder(vectorizer(x))[0]
  top5 = np.linalg.norm(vectors-vec,axis=1).argsort()[:n]
  return [ vocab[x] for x in top5 ]

close_words('paris')

['paris', 'philippines', 'seoul', 'jakarta', 'zoo']

In [112]:
close_words('china')

['china', 'russia', 'pakistan', 'israel', 'turkey']

In [113]:
close_words('official')

['official', 'military', 'office', 'police', 'sources']

## মূল বিষয়

CBoW-এর মতো বুদ্ধিদীপ্ত কৌশল ব্যবহার করে আমরা Word2Vec মডেল প্রশিক্ষণ দিতে পারি। আপনি চাইলে skip-gram মডেলও প্রশিক্ষণ দিতে পারেন, যা কেন্দ্রীয় শব্দটি দিয়ে পার্শ্ববর্তী শব্দ অনুমান করতে শেখে, এবং এটি কতটা ভালো কাজ করে তা দেখতে পারেন।



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদ প্রদানের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা তার জন্য দায়ী থাকব না।
